In [ ]:
len(Quebras.columns)

In [ ]:
QuebrasFinal = Quebras.copy()

In [ ]:
kk

In [ ]:
# Selecionar apenas as colunas numéricas
colunas_numericas = QuebrasFinal.select_dtypes(include=['float64', 'int64']).columns

# 1. Estandardização
scaler = StandardScaler()
QuebrasFinal_scaled = scaler.fit_transform(QuebrasFinal[colunas_numericas])

# 2. Imputação por Regressão
imputer = IterativeImputer()
QuebrasFinal_imputed = imputer.fit_transform(QuebrasFinal_scaled)

# Converter de volta para DataFrame, se necessário
QuebrasFinal_imputed = pd.DataFrame(QuebrasFinal_imputed, columns=colunas_numericas)

QuebrasFinal_imputed

In [ ]:
sns.set(style="whitegrid")

# Plotar violin plot para todas as colunas
plt.figure(figsize=(16, 10))
sns.violinplot(data=QuebrasFinal_imputed, inner="points", orient="h")
plt.title("Violin Plot para Todas as Colunas")
plt.xlabel("Valores")
plt.ylabel("Colunas")
plt.show()

In [ ]:
QuebrasFinal = QuebrasFinal[QuebrasFinal["aprovado"]==0]

In [ ]:
# 1. Estandardização
scaler = StandardScaler()
QuebrasFinal_scaled = scaler.fit_transform(QuebrasFinal)

# 2. Imputação por Regressão
imputer = IterativeImputer()
QuebrasFinal_imputed = imputer.fit_transform(QuebrasFinal_scaled)

# Converter de volta para DataFrame, se necessário
QuebrasFinal_imputed = pd.DataFrame(QuebrasFinal_imputed, columns=QuebrasFinal.columns)

QuebrasFinal_imputed

In [ ]:
sns.set(style="whitegrid")

# Plotar violin plot para todas as colunas
plt.figure(figsize=(16, 10))
sns.violinplot(data=QuebrasFinal_imputed, inner="points", orient="h")
plt.title("Violin Plot para Todas as Colunas")
plt.xlabel("Valores")
plt.ylabel("Colunas")
plt.savefig("Apagar.png")
plt.show()

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt

# Valores de k que você deseja testar
k_values = range(2, 7)

# Lista para armazenar os valores de silhueta média
silhouette_scores = []

# Iterar sobre os valores de k
for k in k_values:
    # Criar o modelo de clustering K-Means
    kmeans = KMeans(n_clusters=k, random_state=42)
    # Treinar o modelo nos dados
    kmeans.fit(QuebrasFinal_imputed)
    # Obter as etiquetas de cluster para cada ponto
    labels = kmeans.labels_
    # Calcular o score de silhueta
    silhouette_avg = silhouette_score(QuebrasFinal_imputed, labels)
    # Armazenar o score de silhueta na lista
    silhouette_scores.append(silhouette_avg)

# Plotar o gráfico do Método do Cotovelo
plt.plot(k_values, silhouette_scores, 'bx-')
plt.xlabel('Número de Clusters (k)')
plt.ylabel('Silhueta Média')
plt.title('Método da Silhueta para Determinar k')
plt.show()

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

# Reduzir a dimensionalidade dos dados para 2 dimensões usando t-SNE
tsne = TSNE(n_components=2, random_state=69)
data_2d = tsne.fit_transform(QuebrasFinal_imputed)

# Plotar os pontos em um gráfico de dispersão, colorindo-os de acordo com os clusters
plt.scatter(data_2d[:, 0], data_2d[:, 1], c=labels, cmap='viridis')
plt.title('Visualização dos Clusters com t-SNE')
plt.xlabel('Componente 1')
plt.ylabel('Componente 2')
plt.colorbar(label='Cluster')
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import dendrogram
from sklearn.datasets import make_blobs
from sklearn.cluster import AgglomerativeClustering

# Gerar dados de exemplo
X, _ = make_blobs(n_samples=150, centers=5, random_state=42)

# Executar o algoritmo de clustering hierárquico
model = AgglomerativeClustering(distance_threshold=0, n_clusters=None)
model.fit(X)

# Função para plotar o dendrograma
def plot_dendrogram(model, **kwargs):
    # Calcula o layout do dendrograma
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # folha
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack([model.children_, model.distances_, counts]).astype(float)

    # Plota o dendrograma
    dendrogram(linkage_matrix, **kwargs)

# Plotando o dendrograma
plt.figure(figsize=(10, 5))
plt.title('Dendrograma Hierárquico')
plot_dendrogram(model)
plt.xlabel('Índice do Amostra')
plt.ylabel('Distância')
plt.grid(False)
plt.savefig("Apagar.png")
plt.show()

In [ ]:
from sklearn.cluster import AgglomerativeClustering

# Inicialize o modelo Agglomerative Clustering com o número desejado de clusters
n_clusters = 3  # Defina o número de clusters desejado
agg_clustering = AgglomerativeClustering(n_clusters=n_clusters)

# Ajuste o modelo aos dados
agg_clustering.fit(QuebrasFinal_imputed)

# Obtenha as etiquetas de cluster para cada amostra
cluster_labels = agg_clustering.labels_

# Adicione as etiquetas de cluster ao DataFrame QuebrasFinal_imputed
QuebrasFinal_imputed['Cluster_Agglomerative'] = cluster_labels

In [ ]:
from sklearn.metrics import silhouette_score, silhouette_samples
import matplotlib.pyplot as plt
import numpy as np

# Calcule a silhouette média para o modelo
silhouette_avg = silhouette_score(QuebrasFinal_imputed, cluster_labels)
print("Silhouette média para o modelo Agglomerative Clustering:", silhouette_avg)

# Calcule a silhouette para cada amostra
sample_silhouette_values = silhouette_samples(QuebrasFinal_imputed, cluster_labels)

# Crie um gráfico de barras para mostrar a silhouette para cada amostra
plt.figure(figsize=(10, 6))
y_lower = 10
for i in range(n_clusters):
    # Agregue a silhouette para amostras pertencentes ao cluster i
    ith_cluster_silhouette_values = sample_silhouette_values[cluster_labels == i]
    ith_cluster_silhouette_values.sort()
    size_cluster_i = ith_cluster_silhouette_values.shape[0]
    y_upper = y_lower + size_cluster_i

    color = plt.cm.nipy_spectral(float(i) / n_clusters)
    plt.fill_betweenx(np.arange(y_lower, y_upper),
                      0, ith_cluster_silhouette_values,
                      facecolor=color, edgecolor=color, alpha=0.7)

    # Etiqueta do cluster no meio
    plt.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

    # Calcule o próximo y_lower para o próximo gráfico
    y_lower = y_upper + 10  

plt.title("Gráfico de Silhouette para Agglomerative Clustering")
plt.xlabel("Valor de silhouette")
plt.ylabel("Cluster")
plt.axvline(x=silhouette_avg, color="red", linestyle="--")  # Linha para indicar a silhouette média
plt.yticks([])  # Limpe os yticks
plt.show()


In [ ]:
QuebrasFinal_imputed

In [ ]:
# Agrupar os dados por cluster e calcular as médias
cluster_means = QuebrasFinal_imputed.groupby('Cluster_Agglomerative').mean()

# Definir cores para os clusters
colors = ['blue', 'green', 'orange', 'red']

# Plotar os gráficos de barras para cada cluster
for i, cluster in enumerate(cluster_means.index):
    cluster_data = cluster_means.loc[cluster]
    plt.figure(figsize=(10, 6))
    plt.barh(cluster_data.index, cluster_data.values, color=colors[i])
    plt.title(f'Médias das Variáveis para o Cluster {cluster}')
    plt.xlabel('Média')
    plt.ylabel('Variável')
    plt.grid(axis='x', linestyle='--', alpha=0.7)
    plt.show()

In [ ]:
# Supondo que 'clusters' seja a coluna de clusters em QuebrasFinal_imputed
clusters = QuebrasFinal_imputed['Cluster_Agglomerative']

# Adicionando a coluna de clusters ao DataFrame Quebras
Quebras['Cluster_Agglomerative'] = clusters

In [ ]:
# Agrupar os dados por causa do incidente e cluster e contar o número de ocorrências em cada grupo
causa_cluster_counts = Quebras.groupby(['causa_do_incidente', 'Cluster_Agglomerative']).size().unstack(fill_value=0)

# Plotar o gráfico de barras empilhadas para cada causa do incidente
causa_cluster_counts.plot(kind='bar', stacked=True, figsize=(16, 12))
plt.title('Quantidade de Incidentes por Causa e Cluster')
plt.xlabel('Causa do Incidente')
plt.ylabel('Quantidade')
plt.xticks(rotation=45, ha='right')
plt.legend(title='Cluster')
plt.show()

In [ ]:
# Agrupar os dados por 'nt' e 'Cluster_Agglomerative' e contar o número de ocorrências em cada grupo
nt_cluster_counts = Quebras.groupby(['nt', 'Cluster_Agglomerative']).size().unstack(fill_value=0)

# Calcular a porcentagem de ocorrências de cada cluster para cada valor de 'nt'
nt_cluster_percentages = nt_cluster_counts.div(nt_cluster_counts.sum(axis=1), axis=0) * 100

# Plotar gráficos circulares para a porcentagem de clusters em cada valor de 'nt'
for nt_value, nt_cluster_percentage in nt_cluster_percentages.iterrows():
    fig, ax = plt.subplots(figsize=(8, 6))
    ax.pie(nt_cluster_percentage, labels=nt_cluster_percentage.index, autopct='%1.1f%%', startangle=90)
    ax.set_title(f'Porcentagem de Clusters em "nt" = {nt_value}')
    plt.tight_layout()
    plt.savefig(f'Porcentagem_Clusters_nt_{nt_value}.svg')
    plt.show()


In [ ]:
# Criar a tabela pivô
pivot_table = Quebras.pivot_table(index='Concelho', columns='Cluster_Agglomerative', aggfunc='size', fill_value=0)

# Renomear as colunas
pivot_table.columns = ['Cluster_' + str(col) for col in pivot_table.columns]

# Exibir a tabela pivô
pivot_table
pivot_table['Cluster_2.0'].idxmax()

In [ ]:
file_path = shape_folder / "concelhos.shp"
Concelho = gpd.read_file(file_path)
Concelho.rename(columns={'NAME_1': 'Distrito'}, inplace=True)
Concelho.rename(columns={'NAME_2': 'Concelho'}, inplace=True)
Concelho["Concelho"] = Concelho["Concelho"].str.title()
Concelho = Concelho[~Concelho['Distrito'].isin(['Azores', 'Madeira'])]
Concelho

In [ ]:
Bilada = set(pivot_table.index) - set(Concelho["Concelho"])
Bilada

In [ ]:
Freguesias_com_clusters = Concelho.merge(pivot_table, left_on='Concelho', right_index=True, how="left")
Freguesias_com_clusters.fillna(0, inplace=True)
Freguesias_com_clusters

In [ ]:
# Configurar a figura e os eixos para a grade de subplots
fig, axs = plt.subplots(1, 3, figsize=(20, 8))

# Iterar sobre cada cluster e plotar o mapa correspondente
for i, cluster in enumerate(['Cluster_0.0', 'Cluster_1.0', 'Cluster_2.0']):
    # Plotar o mapa das freguesias para o cluster atual
    Freguesias_com_clusters.plot(column=cluster, cmap='RdYlGn_r', linewidth=0.8, edgecolor='black', ax=axs[i], legend=True)
    
    # Adicionar título para cada subplot
    axs[i].set_title(f'Mapa do Cluster {i}', fontsize=14)

# Exibir os mapas
plt.tight_layout()
plt.show()


In [ ]:
# Configurar a figura e o eixo
fig, ax = plt.subplots(figsize=(16, 14))

# Plotar o mapa das freguesias
Freguesias_com_clusters.plot(column='Cluster_0.0', cmap = "RdYlGn_r", linewidth=0.8, edgecolor='black', ax=ax, legend=True)

# Adicionar título
ax.set_title('Zonas das Freguesias', fontsize=16)

# Exibir o mapa
plt.show()

In [ ]:
# Configurar a figura e o eixo
fig, ax = plt.subplots(figsize=(16, 14))

# Plotar o mapa das freguesias
Freguesias_com_clusters.plot(column='Cluster_1.0', cmap = "RdYlGn_r", linewidth=0.8, edgecolor='black', ax=ax, legend=True)

# Adicionar título
ax.set_title('Zonas das Freguesias', fontsize=16)

# Exibir o mapa
plt.show()

In [ ]:
# Configurar a figura e o eixo
fig, ax = plt.subplots(figsize=(16, 14))

# Plotar o mapa das freguesias
Freguesias_com_clusters.plot(column='Cluster_2.0', cmap = "RdYlGn_r", linewidth=0.8, edgecolor='black', ax=ax, legend=True)

# Adicionar título
ax.set_title('Zonas das Freguesias', fontsize=16)

plt.savefig("Apagar.png")

# Exibir o mapa
plt.show()